### Introduction

This web page is designed for computing and visualizing for mortgage related information. It only intends for education use **not** for mortgage advice, for professional opinions please consult a real-estate agent.

The statistics we are interested are:

+ the initial payment breakdown
+ the recurrent cost/income breakdown and the cash flow
+ the overall deal summary


In [2]:
from IPython.display import display, HTML

HTML('<script id="infogram_0__/YgtSwWX0p7oqj2sMCneM"title="Average rates on mortgages in 2019" src="https://e.infogram.com/js/dist/embed.js?sIk" type="text/javascript"></script>')

### Parameters that we can automatically compute

+ `mortgage rate` based on Bankrate Data, 2020's **30-year-fixed** will probably be round $4\%$

In [4]:
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import nbinteract as nbi
from ipywidgets import interact

M, K, months, days = 10**6, 10**3, 12, 30

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

def dollar(s):
    return '${:,.2f}'.format(s)

def rmb(s):
    return '￥{:,.2f}'.format(s * 7)

def month_growth(interest):
    return 1.0 + interest / months

def monthly_payment(amount, interest, loan_months):
    payment = float(amount)*interest / (
        float(months)*(1.0-(1.0/month_growth(interest))**loan_months))
    return payment

def basic_report(
    price, 
    down, 
    closing_cost=42830,
    mortgage = 7000,
    loan_term=30,
    num_units=3,
    unit_income=2500,
    repair=50*K,
    insurance=480,
    property_tax=345,
    annual_rate=2/100):
    """
    use https://smartasset.com/mortgage/closing-costs to compute closing-costs
    """
#     try:
    closing_cost, repair, unit_income, insurance, property_tax, purchase_price, down_payment_rate = \
        int(closing_cost), int(repair), int(unit_income), int(insurance), \
        int(property_tax), int(float(price)), down / 100

    vacancy_rate = 5 / 100
    repair_rate = 5 / 100
    property_management_rate = 5 / 100
    
    mortgage_rate = 3.24/100 if loan_term == 15 else 4.0/100
    if not mortgage:
        mortgage = monthly_payment(price, interest=mortgage_rate, loan_months=loan_term*12)
    down_payments, loan = purchase_price * down_payment_rate, purchase_price * (1 - down_payment_rate)
    pre_rent_holding_cost = mortgage * 2

    total_cost = purchase_price+closing_cost+pre_rent_holding_cost+repair
    total_cash_needed = total_cost - loan # or down-payment, closing-cost, pre-rent-holding, repair
    cash_table = pd.DataFrame({
        "expense type (one time)": ["Down Payment", "Closing Cost", "Pre-Rent Holding", "Repair", "(Total)"],
        "amount" : [dollar(down_payments), dollar(closing_cost), dollar(pre_rent_holding_cost), 
                        dollar(repair), dollar(total_cash_needed)]
    })
    display(HTML(cash_table.to_html()))
    total_monthly_income = num_units * unit_income

    vacancy = total_monthly_income * vacancy_rate
    repair = total_monthly_income * repair_rate
    property_management = total_monthly_income * property_management_rate

    estimated_monthly_expense = mortgage + \
    property_tax + insurance + vacancy + repair + property_management
    expense_table = pd.DataFrame({
        "expense type (recurrent)": ["Mortgage", "Repair", "Insurance", "Property tax", "Vacancy", "Property Managment", "(Total)"],
        "amount" : [dollar(mortgage), dollar(repair), dollar(insurance), 

                    dollar(property_tax), dollar(vacancy), dollar(property_management),
                        dollar(estimated_monthly_expense)]
    })
    display(HTML(expense_table.to_html()))
    monthly_cash_flow = total_monthly_income - estimated_monthly_expense
    ROI = monthly_cash_flow * months / total_cash_needed
    
    deal_summary = pd.DataFrame(
        {
            "deal summary": [
                "cash needed in dollar",
                "cash need in rmb",
                "cash flow (net income)",
                "gross income",
                "estimated expense (month)",
                "percentage covered",
                "ROI"
            ],
            "info" : [
                dollar(total_cash_needed),
                rmb(total_cash_needed),
                dollar(monthly_cash_flow),
                dollar(total_monthly_income),
                dollar(estimated_monthly_expense),
                str(round(total_monthly_income / estimated_monthly_expense, 2)*100) + "%",
                str(round(ROI * 100, 2)) + "%"
                       ]}
    )
    display(HTML(deal_summary.to_html()))
    
    # refactoring in progress
    """
    values = []
    # TODO convert this into a parameter
    force_appreciation_rate = 10/100
    for i in range(1, 11):
        __price = int(price * annual_rate ** i) 
        price_after_forced_appreciation = __price * force_appreciation_rate
        print(f"year: {i} price : {'${:,.2f}'.format(price_after_forced_appreciation)}")    
        values.append(price_after_forced_appreciation)

    count = 0
    sales_expenses = []
    for property_value in values:
        total_sales_expense = property_value * 6/100
        print(f"selling after {count} years", '${:,.2f}'.format(total_sales_expense))
        count += 1
        sales_expenses.append(total_sales_expense)

    for i in range(10):
        print(f"year afterwards: {i} year")
        print(f"value: {dollar(values[i])}, \
              sales expense : {dollar(sales_expenses[i])}, \
              loan : {dollar(loans_year_after_year[i])}")
        appreciation_profit = values[i] - sales_expenses[i] - loans_year_after_year[i] - total_cash_needed
        print(f"appreciation_profit: {dollar(appreciation_profit)}")
        cash_flow = monthly_cash_flow * 12 * (i + 1)
        print(f"total cash flow {dollar(cash_flow)}")
        total_profit = appreciation_profit + cash_flow
        print(f"total profit {dollar(total_profit)}")
        print("\n")
    """
    def x_values(max): return np.arange(0, max)
    def y_values(xs, sd):
        return xs + np.random.normal(0, scale=sd, size=len(xs))

    opts = {
        'xlim': (0, 10),
        'ylim': (0, 55),
        'animation_duration': 250,
    }

    return nbi.line(x_values, y_values, max=(10, 50), sd=(1, 10), options=opts)

#     except:
#         print(e)
#         print("Please pass in valid values!")

_ = interact(
    basic_report, 
    price="1800000",
    loan_term={'15-year-fixed': 15, "30-year-fixed": 30},
    down=(5, 50, 5),
    closing_cost = "40000",
    num_units = (1, 12, 1),
    repair = "50000",
    unit_income = "2500",
    insurance = "500",
    property_tax = "400")

interactive(children=(Text(value='1800000', description='price'), IntSlider(value=25, description='down', max=…